In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Simulate Data


Given a field name, and first 2 dimensional numbes, simulate tensor.

In [2]:

from fieldnn.utils.simulate import get_next_info, get_simulated_tensor_from_fldname
import torch

# Real Tensor

In [3]:
def parse_full_recfldgrn(full_recfldgrn):
    # suffix = full_recfldgrn.split('_')[-1]
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    return prefix_ids, rec, fld, grn, suffix

In [4]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

In [5]:
recfldgrn_list = [
                  'A1C@V-A1CNumeDftGrn',
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  'PNSectSent@Sentence-Tk@TknzLLMGrn'
                ]

In [6]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


# Expander Core

Expander try to convert GRN_idx (or GRN_wgt) to embedding vectors.

It is from idx to vector.

It is a type of order-increase. 


Here the core include: CateEmbedding, NumeEmbedding, and LLMEmbed. 



## CateEmbedding (Expander)

### Module

In [7]:
# fieldlm.nn.embedding
from fieldnn.nn.embedding.catembedding import CatEmbeddingLayer

### Config

In [8]:
#######################
embed_size = 512
init = 'random'
#######################

In [9]:
from fieldnn.configfn.expanderfn import get_recfldgrn_embedding

### Usage 

#### recfldgrn information

In [10]:
import pandas as pd

# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-Diag@Value-DiagDftGrn_idx'

prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn

In [11]:
# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
# TODO: also adding padding to v2idx
vocab_size = len(v2idx)
print(vocab_size)

801


#### get configuration

In [12]:
embed_para = get_recfldgrn_embedding(recfldgrn, embed_size, vocab_size, init)
print(embed_para)

{'embedding_size': 512, 'init': 'random', 'vocab_size': 801}


#### init model

In [13]:
if suffix == 'idx':
    NN = CatEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

CatEmbeddingLayer(
  (embedding): Embedding(801, 512, padding_idx=0)
)


#### prepare input

In [14]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information


#### I-NN-O

In [15]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info = info * holder_wgt.unsqueeze(-1)
    print(info.shape, '<----- updated by holder_wgt')
    
    

torch.Size([4, 25, 22, 3]) <----- holder shape
torch.Size([4, 25, 22, 3, 512]) <----- info shape


## NumeEmbedding (Expander)

### Module

In [16]:
# fieldlm.nn.embedding
from fieldnn.nn.embedding.numembedding import NumEmbeddingLayer

### Config

In [17]:
# same as CateEmbedding

### Usage

#### recfldgrn information

In [18]:
# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt'
prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn


# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
# TODO: also adding padding to v2idx
vocab_size = len(v2idx)
print(vocab_size)

37


#### get configuration

In [19]:
embed_para = get_recfldgrn_embedding(recfldgrn, embed_size, vocab_size, init)
print(embed_para)

{'embedding_size': 512, 'init': 'random', 'vocab_size': 37}


#### init model

In [20]:
if suffix == 'idx':
    NN = CatEmbeddingLayer(**embed_para)
    print(NN)
elif suffix == 'wgt':
    NN = NumEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

NumEmbeddingLayer(
  (embedding): Embedding(37, 512, padding_idx=0)
)


#### prepare input

In [21]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


In [22]:
holder[0, 0, 0]

tensor([ 0,  0,  0,  0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         0])

In [23]:
 holder_wgt[0, 0, 0]

tensor([ 0.,  0.,  0.,  0., 11.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.])

#### I-NN-O

In [24]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info_wgt = info * holder_wgt.unsqueeze(-1)
    print(info_wgt.shape, '<----- updated by holder_wgt')

torch.Size([4, 25, 1, 37]) <----- holder shape
torch.Size([4, 25, 1, 37, 512]) <----- info shape
torch.Size([4, 25, 1, 37, 512]) <----- updated by holder_wgt


In [25]:
holder[0, 0, 0]

tensor([ 0,  0,  0,  0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         0])

In [26]:
holder_wgt[0, 0, 0]

tensor([ 0.,  0.,  0.,  0., 11.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.])

In [27]:
info[0, 0, 0][:, 0]

tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.5861e-01,
         4.7504e-03,  1.1655e+00,  1.3496e+00,  6.4677e-01,  1.4808e+00,
        -2.9573e-02, -1.4731e-01,  1.4732e+00, -1.1074e+00, -6.9226e-01,
        -4.9935e-01,  1.0634e-02, -3.9562e-01,  1.8149e-01, -2.6261e-01,
        -2.3695e-01,  8.5560e-01,  8.4573e-01,  2.3062e-01, -1.1472e+00,
        -4.4550e-01,  2.0862e-01, -2.4059e-01, -5.5087e-01,  9.9755e-01,
        -5.6289e-01, -1.4958e-01, -3.5014e-01,  1.3507e-03,  1.8485e+00,
        -4.0584e-01,  0.0000e+00], grad_fn=<SelectBackward0>)

In [28]:
info_wgt[0, 0, 0][:, 0]

tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.9447e+00,
         4.7504e-03,  1.1655e+00,  1.3496e+00,  6.4677e-01,  1.4808e+00,
        -2.9573e-02, -1.4731e-01,  1.4732e+00, -1.1074e+00, -6.9226e-01,
        -4.9935e-01,  1.0634e-02, -3.9562e-01,  1.8149e-01, -2.6261e-01,
        -2.3695e-01,  8.5560e-01,  8.4573e-01,  2.3062e-01, -1.1472e+00,
        -4.4550e-01,  2.0862e-01, -2.4059e-01, -5.5087e-01,  9.9755e-01,
        -5.6289e-01, -1.4958e-01, -3.5014e-01,  1.3507e-03,  1.8485e+00,
        -4.0584e-01,  0.0000e+00], grad_fn=<SelectBackward0>)

## LLMEmbedding (Expander)

### Module

In [29]:
# fieldlm.nn.embedding
from fieldnn.nn.embedding.llmembedding import LLMEmbeddingLayer

### Config

In [30]:
from fieldnn.configfn.expanderfn import get_recfldgrn_llmembedding

### Usage

#### recfldgrn information

In [31]:
[i for i in batch_rfg]

['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [32]:
# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx'
prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn


# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

if 'LLM' in full_recfldgrn:
    tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    # TODO: also adding padding to v2idx
    vocab_size = len(tokenizer)
    print(vocab_size)
else:
    v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    # TODO: also adding padding to v2idx
    vocab_size = len(v2idx)
    print(vocab_size)

30522


#### get configuration

In [33]:
init = tokenizer.name_or_path
embed_para = get_recfldgrn_llmembedding(recfldgrn, embed_size, tokenizer, init)
embed_para

{'embedding_size': 512,
 'init': 'bert-base-uncased',
 'tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})}

#### init model

In [34]:
if suffix == 'idx' and 'LLM' not in recfldgrn:
    NN = CatEmbeddingLayer(**embed_para)
    print(NN)
    
elif suffix == 'idx' and 'LLM' in recfldgrn:
    NN = LLMEmbeddingLayer(**embed_para)
    
elif suffix == 'wgt':
    NN = NumEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### prepare input

In [35]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information


#### I-NN-O

In [36]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info_wgt = info * holder_wgt.unsqueeze(-1)
    print(info_wgt.shape, '<----- updated by holder_wgt')

torch.Size([4, 25, 1, 14, 121, 11]) <----- holder shape
torch.Size([4, 25, 1, 14, 121, 11, 512]) <----- info shape


In [37]:
leng_mask = holder == 0
leng_mask.shape

torch.Size([4, 25, 1, 14, 121, 11])

In [38]:
info.shape

torch.Size([4, 25, 1, 14, 121, 11, 512])

In [39]:
# TODO: do the tensor validation

# Expander

## Module

In [40]:
from fieldnn.sublayer.expander import Expander_Layer

## Config

In [41]:
from fieldnn.configfn.expanderfn import get_expander_para

## Usage

### recfldgrn information

In [42]:
full_recfldgrn_list = ['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
                       'B-P-EC-Diag@Value-DiagDftGrn_idx',
                       'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

full_recfldgrn_list

['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [43]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [44]:
full_rfg_to_basicInfo = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) full recfldgrn information
    prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
    recfldgrn = rec + '@' + fld + '-' + grn


    # (2) get vocab information
    fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
    df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

    if 'LLM' in full_recfldgrn:
        tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # TODO: also adding padding to v2idx
        # vocab_size = len(tokenizer)
        # print(vocab_size)
        vocab_tokenizer = tokenizer
        init = tokenizer.name_or_path
    else:
        v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # TODO: also adding padding to v2idx
        vocab_size = len(v2idx)
        # print(vocab_size)
        vocab_tokenizer = vocab_size
        init = 'random'
    
    d = {'vocabsize_tokenizer': vocab_tokenizer, 'init': init}
    full_rfg_to_basicInfo[full_recfldgrn] = d
    
    
full_rfg_to_basicInfo

{'B-P-EC-A1C@V-A1CNumeDftGrn_wgt': {'vocabsize_tokenizer': 37,
  'init': 'random'},
 'B-P-EC-Diag@Value-DiagDftGrn_idx': {'vocabsize_tokenizer': 801,
  'init': 'random'},
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx': {'vocabsize_tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
  'init': 'bert-base-uncased'}}

### get configuration

In [45]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


full_rfg_to_ExpanderConfig = {}

for full_recfldgrn in full_recfldgrn_list:
    
    # determine the nn_name
    if '_idx' in full_recfldgrn and 'LLM' not in full_recfldgrn:
        nn_name = 'CatEmbeddingLayer'

    elif '_idx' in full_recfldgrn and 'LLM' in full_recfldgrn:
        nn_name = 'LLMEmbeddingLayer'

    elif '_wgt' in full_recfldgrn:
        nn_name = 'NumEmbeddingLayer'
    else:
        raise ValueError(f'full_recfldgrn is not correct "{full_recfldgrn}"')
    
    d = full_rfg_to_basicInfo[full_recfldgrn]
    
    # print('\n----')
    # print(full_recfldgrn)
    # print(d)
    # print(nn_name)
    expander_para = get_expander_para(full_recfldgrn, 
                                       nn_name,
                                       embed_size, d['vocabsize_tokenizer'], d['init'], 
                                       process)
    # print(expander_para)
    output_recfld = full_recfldgrn.split('Grn')[0]
    
    full_rfg_to_ExpanderConfig[full_recfldgrn] = {
        'full_recfldgrn': full_recfldgrn,
        'output_recfld': output_recfld,
        'expander_para': expander_para,
    }
    
    
# full_rfg_to_ExpanderConfig

In [46]:
# full_rfg_to_ExpanderConfig

### init model

In [47]:
FullRFG_2_Embed = {}

for full_recfldgrn, ExpanderConfig in full_rfg_to_ExpanderConfig.items():
    
    FullRFG_2_Embed[full_recfldgrn] = Expander_Layer(**ExpanderConfig)
    print(full_recfldgrn)
    print(FullRFG_2_Embed[full_recfldgrn])
    

B-P-EC-A1C@V-A1CNumeDftGrn_wgt
Expander_Layer(
  (Embed): NumEmbeddingLayer(
    (embedding): Embedding(37, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC-Diag@Value-DiagDftGrn_idx
Expander_Layer(
  (Embed): CatEmbeddingLayer(
    (embedding): Embedding(801, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
Expander_Layer(
  (Embed): LLMEmbeddingLayer(
    (LLM): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense):

### prepare input

In [48]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print(full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str: 
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')

    print('\n')

B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information




### I-NN-O

In [49]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print('Input:',full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str:  
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')
        
        
    NN = FullRFG_2_Embed[full_recfldgrn] 
    print(holder.shape, '<----- holder shape')
    output_recfld, holder, info = NN(full_recfldgrn, holder, holder_wgt)
    print('Output:', output_recfld)
    print(info.shape, '<----- info shape')

    print('\n')

Input: B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information
torch.Size([4, 25, 1, 37]) <----- holder shape
Output: B-P-EC-A1C@V-A1CNumeDft
torch.Size([4, 25, 1, 37, 128]) <----- info shape


Input: B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 22, 3]) <----- holder shape
Output: B-P-EC-Diag@Value-DiagDft
torch.Size([4, 25, 22, 3, 128]) <----- info shape


Input: B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 1, 14, 121, 11]) <----- holder shape
Output: B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM
torch.Size([4, 25, 1, 14, 121, 11, 128]) <----- info shape


